<a href="https://colab.research.google.com/github/Oaybarf/FX/blob/main/Prueba_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
import pytz

# Definir la zona horaria de Perú
timezone = pytz.timezone("America/Lima")

# Obtener la fecha y hora actual en Perú
current_time_peru = datetime.now(timezone)

# Crear un DataFrame con la fecha y hora actual
data = {
    "Fecha": [current_time_peru.strftime("%Y-%m-%d")],
    "Hora": [current_time_peru.strftime("%H:%M:%S")]
}

df = pd.DataFrame(data)
print(df)

        Fecha      Hora
0  2024-10-02  11:09:36
